In [1]:
import logging; logging.basicConfig(level=logging.DEBUG)
import time

import ipywidgets as ipyw
import mr_box_peripheral_board as mrbox

proxy = mrbox.SerialProxy(port='COM14', baudrate=57600, settling_time_s=2.5)
proxy._timeout_s = 20
proxy.zstage.motor_enabled = True
proxy.zstage.home_stop_enabled = True
proxy.zstage.micro_stepping = True

DEBUG:base_node_rpc.proxy:Attempt to connect to device on port COM14 (baudrate=57600)
DEBUG:serial_device.threaded:Open `COM14` and monitor connection status
DEBUG:serial_device.threaded:connection_made: `COM14` `<ReaderThread(Thread-7, started daemon 5860)>`
DEBUG:base_node_rpc.proxy:Wait for connection to port COM14
INFO:base_node_rpc.proxy:Successfully connected to mr-box-peripheral-board on port COM14


In [3]:
def zstage_move_to(position):
    proxy.zstage.motor_enabled = True
    proxy.zstage_move_to(position)
    proxy.zstage.motor_enabled = False
    

zstage_range = ipyw.FloatRangeSlider(min=0, max=11, step=.1, value=(0, 10.1),
                                    description='position range:')
def zstage_range_changed(message):
    if message['old'][0] == message['new'][0]:
        # Range **max** has changed.
        if zstage_state.value == 'up':
            # Magnet is in "up" position.  Move magnet to new range max position.
            zstage_move_to(message['new'][1])
    else:
        # Range **min** has changed.
        if zstage_state.value in 'down':
            # Magnet is in "down" position.  Move magnet to new range min position.
            zstage_move_to(message['new'][0])
zstage_range.observe(zstage_range_changed, names=['value'])

def _zstage_home(*args):
    home_retry_count = 10
    proxy.zstage.motor_enabled = True
    try:
        for i in xrange(home_retry_count):
            proxy.zstage_home()
            time.sleep(1.5)
            if proxy.zstage_at_home():
                break
        else:
            raise RuntimeError('Error homing after %s attempts.' %
                               home_retry_count)
    finally:
        proxy.zstage.motor_enabled = True
    zstage_state.value = 'down'

zstage_home = ipyw.Button(description='Home')
zstage_home.on_click(_zstage_home)

def _zstage_state(message):
    min_, max_ = zstage_range.value
    if message['new'] == 'up':
        # Engage magnet.
        position = max_
        zstage_state.value = 'up'
    else:
        # Disengage magnet.
        position = min_
        zstage_state.value = 'down'
    zstage_move_to(position)

zstage_state = ipyw.RadioButtons(options=['down', 'up'])
zstage_state.observe(_zstage_state, names=['value'])

_zstage_home()
ipyw.VBox([zstage_home, zstage_range, zstage_state])